<h1 align=center><font size = 5><em></em>Final Report: Where to open my bar ? </font></h1>


### 1. Introduction 

#### 1.1. Background

##### New York city

The City of New York, usually called either New York City (NYC) or simply New York (NY), is the most populous city in the United States. With an estimated 2017 population of 8,622,698 distributed over a land area of about 302.6 square miles (784 km2), New York is also the most densely populated major city in the United States. Located at the southern tip of the state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass and one of the world's most populous megacities, with an estimated 20,320,876 people in its 2017 Metropolitan Statistical Area and 23,876,155 residents in its Combined Statistical Area. A global power city, New York City has been described as the cultural, financial, and media capital of the world, and exerts a significant impact upon commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports.

#### 1.2. Problem

Let's suppose thaht someone wants to open a new bar in a city (take New York for our example of city). The first reflex that comes is to find where there is a high and low density of Bars to look for a place where he can benifit from his new Bar. For exemple, the majority will choose to open a new Bar where there is no high density of Bars.
In this report, we are going to use you will use the Foursquare API to explore neighborhoods in Toronto. 

We'll group the neighborhoods into clusters and use them to choose where can someone open a new Bar (for exemple) to get the maximum benefits. Of course, it will depend of how many other bars exists in every neighborhoods, which means that if a bar is isolated from the others, it will give more benefits from people who are visiting this neighborhood.

### 2. Data acquisition and cleaning

#### 2.1. Data sources

Using data in our previous lab, we're import it from https://cocl.us/new_york_dataset.
The data we're going to explore is composed 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

Our first data set is composed of 4 columns :
- Borough 
- Neighborhood 
- Latitude
- Longitude
    

Then we'll use Foursquare api to get venues in a radius of 500 m for every neighborhood and do some statistical methods to classify neighborhoods and make a choice of where can this person open a new Bar to fet the maximum benifits from it

New York City map will be used for this project.
The map is on the link: https://geo.nyu.edu/catalog/nyu_2451_34572
Publisher: New York (City). Department of City Planning Place(s)


#### 2.1. Data structuring and cleaning

After getting our data, we can will structure it as a dataframe as showed below :

In [3]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


After getting the data of every Borough and Neighborhood on New York with Their Latitude and Longitude, we'll use Foursquare API to get venues in every Neighborhod. The result of calling the Api is shown here : 

In [8]:
CLIENT_ID = '2NZZJEQR1QROVHP3V5K1VTRORYRNQWLWOTGVM02YI0PFDUAM' # your Foursquare ID
CLIENT_SECRET = 'GA5BRDC3NVZDJJNX13YP0MX5C1BJ1QDZMXXAYANIIZNMBTLH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
LIMIT = 10
radius = 100
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [9]:
newyork_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
5,Wakefield,40.894705,-73.847201,SUBWAY,40.890656,-73.849192,Sandwich Place
6,Wakefield,40.894705,-73.847201,Pitman Deli,40.894149,-73.845748,Food
7,Wakefield,40.894705,-73.847201,Baychester Avenue Food Truck,40.892293,-73.843230,Food Truck
8,Wakefield,40.894705,-73.847201,Koss Quick Wash,40.891147,-73.850230,Laundromat
9,Co-op City,40.874294,-73.829939,Capri II Pizza,40.876374,-73.829940,Pizza Place


## 3. Exploratory Data Analysis


Let's first create a map where we can render the neighborhoods in New York :

In [10]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The map shows all the neighborhoods of New York with all venues. We'll filter our data in the next steps.

We'll group rows by neighborhood and by taking the mean of the frequency of occurrence of each category where we'll find the name 'Bar' on it.
to select columns with 'Bar', we'll use Regex for regular expressions. The final result is shown here :

In [12]:
import sys
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()
newyork_grouped1 = newyork_grouped.filter(regex='Bar')
newyork_grouped1['Neighborhood'] = newyork_grouped['Neighborhood']
newyork_grouped = newyork_grouped1
cols = newyork_grouped.columns.tolist()
cols = cols[-1:] + cols[:-1]
newyork_grouped = newyork_grouped[cols]
newyork_grouped.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Neighborhood,Bar,Beer Bar,Cocktail Bar,Dive Bar,Gay Bar,Hookah Bar,Hotel Bar,Juice Bar,Karaoke Bar,Sake Bar,Salon / Barbershop,Sports Bar,Whisky Bar,Wine Bar
0,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
1,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125,0.0,0.0
2,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
3,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0
4,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0


As you can see, the dataframe show every neighborhood in new york with the frequence of every type of Bars

Now let's create the new dataframe and display the top 10 venues that contain Bars. The final result is shown here : 

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = newyork_grouped['Neighborhood']
for ind in np.arange(newyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(newyork_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
1,Annadale,Sports Bar,Wine Bar,Whisky Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
2,Arden Heights,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
3,Arlington,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
4,Arrochar,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
5,Arverne,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
6,Astoria,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
7,Astoria Heights,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
8,Auburndale,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
9,Bath Beach,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar


Our Data is now ready to fit k-means algorithm that we'll use in the next steps

## 4. Modeling

Let's use k-means to cluster the neighborhood into 5 clusters. It means that the value of k for this exemple is 5.

k-means is one of the simplest and popular unsupervised machine learning algorithms. In other words, the K-means algorithm identifies k number of centroids, and      then allocates every data point to the nearest cluster, while keeping the centroids as small as possible.

Here is the result of clustering with 5 labels after fiting our k-means algorithm :

In [18]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = newyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
manhattan_merged = neighborhoods
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged = manhattan_merged[np.isfinite(manhattan_merged['Cluster Labels'])]
manhattan_merged.head(10) 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
1,Bronx,Co-op City,40.874294,-73.829939,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
2,Bronx,Eastchester,40.887556,-73.827806,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
3,Bronx,Fieldston,40.895437,-73.905643,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
4,Bronx,Riverdale,40.890834,-73.912585,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
5,Bronx,Kingsbridge,40.881687,-73.902818,0.0,Beer Bar,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar
6,Manhattan,Marble Hill,40.876551,-73.910660,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
7,Bronx,Woodlawn,40.898273,-73.867315,3.0,Bar,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar
8,Bronx,Norwood,40.877224,-73.879391,0.0,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar,Gay Bar
9,Bronx,Williamsbridge,40.881039,-73.857446,1.0,Bar,Wine Bar,Whisky Bar,Sports Bar,Salon / Barbershop,Sake Bar,Karaoke Bar,Juice Bar,Hotel Bar,Hookah Bar


## 5. Result

Let's show our clusters on a map using Folium to get a visual version of the clusters :

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
result=[]
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
    result.append(rainbow[int(cluster-1)])

map_clusters

As you can see on the map, we have 5 clusters of every type of bars. This clusters can be labeled 

The result show with different colors 5 clusters depending of the density of Bars in every neighborhood